# Code 10: Probabilistic Programming Languages

In [1]:
from scipy import stats
import pymc as pm
import pytensor
import numpy as np
import matplotlib.pyplot as plt
import arviz as az

import datetime
print(f"Last Run {datetime.datetime.now()}")

Last Run 2023-12-27 17:34:02.444362


In [2]:
az.style.use("arviz-grayscale")
plt.rcParams["figure.dpi"] = 300

## Posterior Computation

### Code 10.1

In [3]:
from jax import grad

simple_grad = grad(lambda x: x**2)
print(simple_grad(4.0))

8.0


### Code 10.2

In [4]:
from jax import grad
from jax.scipy.stats import norm

def model(test_point, observed):
    z_pdf = norm.logpdf(test_point, loc=0, scale=5)
    x_pdf = norm.logpdf(observed, loc=test_point, scale=1)
    logpdf = z_pdf + x_pdf
    return logpdf

model_grad = grad(model)

observed, test_point = 5.0, 2.5 
logp_val = model(test_point, observed)
grad = model_grad(test_point, observed)
print(f"log_p_val: {logp_val}")
print(f"grad: {grad}")

log_p_val: -6.697315216064453
grad: 2.4000000953674316


### Code 10.3

In [5]:
with pm.Model() as model:
    z = pm.Normal("z", 0., 5.)
    x = pm.Normal("x", mu=z, sigma=1., observed=observed)

func = model.logp_dlogp_function()
func.set_extra_values({})
print(func(np.array([test_point])))

(array(-6.69731498), array([2.4]))


### Code 10.4

In [6]:
def fraud_detector(fraud_observations, non_fraud_observations, fraud_prior=8, non_fraud_prior=6):
    """Conjugate Beta Binomial model for fraud detection"""
    expectation = (fraud_prior + fraud_observations) / (
        fraud_prior + fraud_observations + non_fraud_prior + non_fraud_observations)
    
    if expectation > .5:
        return {"suspend_card":True}

%timeit fraud_detector(2, 0)

81.7 ns ± 0.693 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## PPL Driven Transformations

### Code 10.9

In [7]:
observed = np.repeat(2, 2)
pdf = stats.norm(0, 1).pdf(observed)
np.prod(pdf, axis=0)

0.0029150244650281935

### Code 10.10

In [8]:
observed = np.repeat(2, 1000)
pdf = stats.norm(0, 1).pdf(observed)
np.prod(pdf, axis=0)

0.0

### Code 10.11

In [9]:
1.2 - 1

0.19999999999999996

In [10]:
pdf[0], np.prod(pdf, axis=0)

(0.05399096651318805, 0.0)

### Code 10.12

In [11]:
logpdf = stats.norm(0, 1).logpdf(observed)
np.log(pdf[0]), logpdf[0], logpdf.sum()

(-2.9189385332046727, -2.9189385332046727, -2918.9385332046736)

In [12]:
np.log(pdf[0])

-2.9189385332046727

### Distribution Transforms

### Code 10.13

In [13]:
lower, upper = -1, 2
domain = np.linspace(lower, upper, 5)
transform = np.log(domain - lower) - np.log(upper - domain)
print(f"Original domain: {domain}")
print(f"Transformed domain: {transform}")

Original domain: [-1.   -0.25  0.5   1.25  2.  ]
Transformed domain: [       -inf -1.09861229  0.          1.09861229         inf]


/tmp/ipykernel_11005/2568761612.py:3: RuntimeWarning: divide by zero encountered in log
  transform = np.log(domain - lower) - np.log(upper - domain)


### Code 10.14

In [14]:
with pm.Model() as model:
    x = pm.Uniform("x", -1., 2.)
    
model.values_to_rvs

{x_interval__: x ~ Uniform(-1, 2)}

In [15]:
model.varlogp_nojac.eval

<bound method Variable.eval of __logp_nojac>

In [16]:
print(model.varlogp.eval({"x_interval__":-2}),
      model.varlogp_nojac.eval({"x_interval__":-2}))
print(model.varlogp.eval({"x_interval__":1}),
      model.varlogp_nojac.eval({"x_interval__":1}))

-2.2538560220859454 -1.0986122886681098
-1.6265233750364456 -1.0986122886681098


### Code 10.16

In [17]:
import tensorflow_probability as tfp
tfd = tfp.distributions

In [18]:
tfb = tfp.bijectors

lognormal0 = tfd.LogNormal(0., 1.)
lognormal1 = tfd.TransformedDistribution(tfd.Normal(0., 1.), tfb.Exp())
x = lognormal0.sample(100)

np.testing.assert_array_equal(lognormal0.log_prob(x), lognormal1.log_prob(x))

2023-12-27 17:34:10.985938: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 17:34:10.985963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 17:34:10.986582: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 17:34:11.611796: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Code 10.17

In [19]:
y_observed = stats.norm(0, .01).rvs(20)

with pm.Model() as model_transform:
    sd = pm.HalfNormal("sd", 5)
    y = pm.Normal("y", mu=0, sigma=sd, observed=y_observed)
    idata_transform = pm.sample(chains=1, draws=100000)

print(model_transform.values_to_rvs)
print(f"Diverging: {idata_transform.sample_stats['diverging'].sum().item()}")

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sd]


Sampling 1 chain for 1_000 tune and 100_000 draw iterations (1_000 + 100_000 draws total) took 20 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


{sd_log__: sd, TensorConstant(TensorType(float64, shape=(20,)), data=array([-0. ... 01521619])): y}
Diverging: 0


### Code 10.18

In [20]:
with pm.Model() as model_no_transform:
    sd = pm.HalfNormal("sd", 5, transform=None)
    y = pm.Normal("y", mu=0, sigma=sd, observed=y_observed)
    idata_no_transform = pm.sample(chains=1, draws=100000)

print(model_no_transform.values_to_rvs)
print(f"Diverging: {idata_no_transform.sample_stats['diverging'].sum().item()}")

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [sd]


Sampling 1 chain for 1_000 tune and 100_000 draw iterations (1_000 + 100_000 draws total) took 21 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


{sd: sd, TensorConstant(TensorType(float64, shape=(20,)), data=array([-0. ... 01521619])): y}
Diverging: 41


## Operation Graphs and Automatic Reparameterization

In [21]:
x = 3
y = 1
x * y / x + 2

3.0

### Code 10.19

In [22]:
pytensor.config.compute_test_value = 'ignore'

In [23]:
x = pytensor.tensor.vector("x")
y = pytensor.tensor.vector("y")
out = x*(y/x) + 0
pytensor.printing.debugprint(out)

Add [id A]
 ├─ Mul [id B]
 │  ├─ x [id C]
 │  └─ True_div [id D]
 │     ├─ y [id E]
 │     └─ x [id C]
 └─ ExpandDims{axis=0} [id F]
    └─ 0 [id G]


### Code 10.20

In [24]:
fgraph = pytensor.function([x,y], [out])
pytensor.printing.debugprint(fgraph)

DeepCopyOp [id A] 0
 └─ y [id B]


### Code 10.21

In [25]:
fgraph([1],[3])

[array([3.])]

### Figure 10.1 and Figure 10.2

In [26]:
pytensor.printing.pydotprint(
    out, outfile="img/chp10/symbolic_graph_unopt.png",
    var_with_name_simple=False, high_contrast=False, with_ids=True)
pytensor.printing.pydotprint(
    fgraph, 
    outfile="img/chp10/symbolic_graph_opt.png", 
    var_with_name_simple=False, high_contrast=False, with_ids=True)

The output file is available at img/chp10/symbolic_graph_unopt.png
The output file is available at img/chp10/symbolic_graph_opt.png


### Code 10.22

In [27]:
with pm.Model() as model_normal:
    x = pm.Normal("x", 0., 1.)
    
pytensor.printing.debugprint(model_normal.logp())

Sum{axes=None} [id A] '__logp'
 └─ MakeVector{dtype='float64'} [id B]
    └─ Sum{axes=None} [id C]
       └─ Check{sigma > 0} [id D] 'x_logprob'
          ├─ Sub [id E]
          │  ├─ Sub [id F]
          │  │  ├─ Mul [id G]
          │  │  │  ├─ -0.5 [id H]
          │  │  │  └─ Pow [id I]
          │  │  │     ├─ True_div [id J]
          │  │  │     │  ├─ Sub [id K]
          │  │  │     │  │  ├─ x [id L]
          │  │  │     │  │  └─ 0.0 [id M]
          │  │  │     │  └─ 1.0 [id N]
          │  │  │     └─ 2 [id O]
          │  │  └─ Log [id P]
          │  │     └─ Sqrt [id Q]
          │  │        └─ 6.283185307179586 [id R]
          │  └─ Log [id S]
          │     └─ 1.0 [id N]
          └─ All{axes=None} [id T]
             └─ MakeVector{dtype='bool'} [id U]
                └─ All{axes=None} [id V]
                   └─ Gt [id W]
                      ├─ 1.0 [id N]
                      └─ 0 [id X]


## Effect handling

### Code 10.23

In [28]:
import jax
import numpyro
from tensorflow_probability.substrates import jax as tfp_jax

tfp_dist = tfp_jax.distributions
numpyro_dist = numpyro.distributions

root = tfp_dist.JointDistributionCoroutine.Root
def tfp_model():
    x = yield root(tfp_dist.Normal(loc=1.0, scale=2.0, name="x"))
    z = yield root(tfp_dist.HalfNormal(scale=1., name="z"))
    y = yield tfp_dist.Normal(loc=x, scale=z, name="y")
    
def numpyro_model():
    x = numpyro.sample("x", numpyro_dist.Normal(loc=1.0, scale=2.0))
    z = numpyro.sample("z", numpyro_dist.HalfNormal(scale=1.0))
    y = numpyro.sample("y", numpyro_dist.Normal(loc=x, scale=z))

In [29]:
try:
    print(tfp_model())
except:
    pass

<generator object tfp_model at 0x7fbd6eb7ee40>


In [30]:
try:
    print(numpyro_model())
except:
    pass

### Code 10.24

In [31]:
sample_key = jax.random.PRNGKey(52346)

# Draw samples
jd = tfp_dist.JointDistributionCoroutine(tfp_model)
tfp_sample = jd.sample(1, seed=sample_key)

predictive = numpyro.infer.Predictive(numpyro_model, num_samples=1)
numpyro_sample = predictive(sample_key)

# Evaluate log prob
log_likelihood_tfp = jd.log_prob(tfp_sample)
log_likelihood_numpyro = numpyro.infer.util.log_density(
    numpyro_model, [], {},
    # Samples returning from JointDistributionCoroutine is a
    # Namedtuple like Python object, we convert it to a dictionary
    # so that numpyro can recognize it.
    params=tfp_sample._asdict())

# Validate that we get the same log prob
np.testing.assert_allclose(log_likelihood_tfp, log_likelihood_numpyro[0], rtol=1e-6)

### Code 10.25

In [32]:
# Condition z to .01 in TFP and sample
jd.sample(z=.01, seed=sample_key)

StructTuple(
  x=Array(0.4250738, dtype=float32),
  z=Array(0.01, dtype=float32),
  y=Array(0.42213488, dtype=float32)
)

In [33]:
# Condition z to .01 in NumPyro and sample
predictive = numpyro.infer.Predictive(
    numpyro_model, num_samples=1, params={"z": np.asarray(.01)})
predictive(sample_key)

{'x': Array([1.1232406], dtype=float32),
 'y': Array([1.1318898], dtype=float32),
 'z': array([0.01])}

### Code 10.26

In [34]:
# Conditioned z to .01 in TFP and construct conditional distributions
dist, value = jd.sample_distributions(z=.01, seed=sample_key)
assert dist.y.loc == value.x
assert dist.y.scale == value.z

# Conditioned z to .01 in NumPyro and construct conditional distributions
model = numpyro.handlers.substitute(numpyro_model, data={"z": .01})
with numpyro.handlers.seed(rng_seed=sample_key):
    # Under the seed context, the default behavior of a NumPyro model is the
    # same as in Pyro: drawing prior sample.
    model_trace = numpyro.handlers.trace(numpyro_model).get_trace()
assert model_trace["y"]["fn"].loc == model_trace["x"]["value"]
assert model_trace["y"]["fn"].scale == model_trace["z"]["value"]

## Designing a PPL

### Code 10.27

In [35]:
import numpy as np
from scipy import stats

# Draw 2 samples from a Normal(1., 2.) distribution
x = stats.norm.rvs(loc=1.0, scale=2.0, size=2, random_state=1234)
# Evaluate the log probability of the samples 
logp = stats.norm.logpdf(x, loc=1.0, scale=2.0)

### Code 10.28

In [36]:
random_variable_x = stats.norm(loc=1.0, scale=2.0)

x = random_variable_x.rvs(size=2, random_state=1234)
logp = random_variable_x.logpdf(x)

### Code 10.29

In [37]:
x = stats.norm(loc=1.0, scale=2.0)
y = stats.norm(loc=x, scale=0.1)
y.rvs()

TypeError: unsupported operand type(s) for +: 'float' and 'rv_continuous_frozen'

### Code 10.30

In [38]:
class RandomVariable:
    def __init__(self, distribution):
        self.distribution = distribution

    def __array__(self):
        return np.asarray(self.distribution.rvs())

x = RandomVariable(stats.norm(loc=1.0, scale=2.0))
z = RandomVariable(stats.halfnorm(loc=0., scale=1.))
y = RandomVariable(stats.norm(loc=x, scale=z))

for i in range(5):
    print(np.asarray(y))

1.8371904749402057
0.9115299858011244
1.1045295605002439
0.8569527358843754
-1.3368563634713648


### Code 10.31

In [39]:
class RandomVariable:
    def __init__(self, distribution, value=None):
        self.distribution = distribution
        self.set_value(value)
    
    def __repr__(self):
        return f"{self.__class__.__name__}(value={self.__array__()})"

    def __array__(self, dtype=None):
        if self.value is None:
            return np.asarray(self.distribution.rvs(), dtype=dtype)
        return self.value

    def set_value(self, value=None):
        self.value = value
    
    def log_prob(self, value=None):
        if value is not None:
            self.set_value(value)
        return self.distribution.logpdf(np.array(self))

x = RandomVariable(stats.norm(loc=1.0, scale=2.0))
z = RandomVariable(stats.halfnorm(loc=0., scale=1.))
y = RandomVariable(stats.norm(loc=x, scale=z))

### Code 10.32

In [40]:
for i in range(3):
    print(y)

print(f"  Set x=5 and z=0.1")
x.set_value(np.asarray(5))
z.set_value(np.asarray(0.05))
for i in range(3):
    print(y)

print(f"  Reset z")
z.set_value(None)
for i in range(3):
    print(y)

RandomVariable(value=1.3005379430504267)
RandomVariable(value=3.5547158349681687)
RandomVariable(value=-1.584682011128197)
  Set x=5 and z=0.1
RandomVariable(value=4.931518401510109)
RandomVariable(value=5.008135750884229)
RandomVariable(value=4.9915659170051265)
  Reset z
RandomVariable(value=4.995529441560821)
RandomVariable(value=3.35689470946334)
RandomVariable(value=5.279468777072529)


### Code 10.33

In [41]:
# Observed y = 5.
y.set_value(np.array(5.))

posterior_density = lambda xval, zval: x.log_prob(xval) + z.log_prob(zval) + \
                y.log_prob()
posterior_density(np.array(0.), np.array(1.))

-15.881815599614018

### Code 10.34

In [42]:
def log_prob(xval, zval, yval=5):
    x_dist = stats.norm(loc=1.0, scale=2.0)
    z_dist = stats.halfnorm(loc=0., scale=1.)
    y_dist = stats.norm(loc=xval, scale=zval)
    return x_dist.logpdf(xval) + z_dist.logpdf(zval) + y_dist.logpdf(yval)

log_prob(0, 1)

-15.881815599614018

### Code 10.35

In [43]:
def prior_sample():
    x = stats.norm(loc=1.0, scale=2.0).rvs()
    z = stats.halfnorm(loc=0., scale=1.).rvs()
    y = stats.norm(loc=x, scale=z).rvs()
    return x, z, y

prior_sample()

(2.390718768333718, 0.5995096105486906, 2.859734598142775)

### Shape handling

### Code 10.36

In [44]:
def prior_sample(size):
    x = stats.norm(loc=1.0, scale=2.0).rvs(size=size)
    z = stats.halfnorm(loc=0., scale=1.).rvs(size=size)
    y = stats.norm(loc=x, scale=z).rvs()
    return x, z, y

print([x.shape for x in prior_sample(size=(2))])
print([x.shape for x in prior_sample(size=(2, 3, 5))])

[(2,), (2,), (2,)]
[(2, 3, 5), (2, 3, 5), (2, 3, 5)]


### Code 10.37

In [45]:
n_row, n_feature = 1000, 5
X = np.random.randn(n_row, n_feature)

def lm_prior_sample0():
    intercept = stats.norm(loc=0, scale=10.0).rvs()
    beta = stats.norm(loc=np.zeros(n_feature), scale=10.0).rvs()
    sigma = stats.halfnorm(loc=0., scale=1.).rvs()
    y_hat = X @ beta + intercept
    y = stats.norm(loc=y_hat, scale=sigma).rvs()
    return intercept, beta, sigma, y

def lm_prior_sample(size=10):
    if isinstance(size, int):
        size = (size,)
    else:
        size = tuple(size)
    intercept = stats.norm(loc=0, scale=10.0).rvs(size=size)
    beta = stats.norm(loc=np.zeros(n_feature), scale=10.0).rvs(
        size=size + (n_feature,))
    sigma = stats.halfnorm(loc=0., scale=1.).rvs(size=size)
    y_hat = np.squeeze(X @ beta[..., None]) + intercept[..., None]
    y = stats.norm(loc=y_hat, scale=sigma[..., None]).rvs()
    return intercept, beta, sigma, y

In [46]:
print([x.shape for x in lm_prior_sample0()])

[(), (5,), (), (1000,)]


In [47]:
print([x.shape for x in lm_prior_sample(size=())])
print([x.shape for x in lm_prior_sample(size=10)])
print([x.shape for x in lm_prior_sample(size=[10, 3])])

[(), (5,), (), (1000,)]
[(10,), (10, 5), (10,), (10, 1000)]
[(10, 3), (10, 3, 5), (10, 3), (10, 3, 1000)]


In [48]:
# def lm_prior_sample2(size=10):
#     intercept = stats.norm(loc=0, scale=10.0).rvs(size=size)
#     beta = stats.multivariate_normal(
#         mean=np.zeros(n_feature), cov=10.0).rvs(size=size)
#     sigma = stats.halfnorm(loc=0., scale=1.).rvs(size=size)
#     y_hat = np.einsum('ij,...j->...i', X, beta) + intercept[..., None]
#     y = stats.norm(loc=y_hat, scale=sigma[..., None]).rvs()
#     return intercept, beta, sigma, y

# print([x.shape for x in lm_prior_sample2(size=())])
# print([x.shape for x in lm_prior_sample2(size=10)])
# print([x.shape for x in lm_prior_sample2(size=(10, 3))])

### Code 10.38

In [49]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

jd = tfd.JointDistributionSequential([
    tfd.Normal(0, 10),
    tfd.Sample(tfd.Normal(0, 10), n_feature),
    tfd.HalfNormal(1),
    lambda sigma, beta, intercept: tfd.Independent(
        tfd.Normal(
            loc=tf.einsum("ij,...j->...i", X, beta) + intercept[..., None],
            scale=sigma[..., None]),
        reinterpreted_batch_ndims=1,
        name="y")
])

print(jd)

n_sample = [3, 2]
for log_prob_part in jd.log_prob_parts(jd.sample(n_sample)):
    assert log_prob_part.shape == n_sample

tfp.distributions.JointDistributionSequential("JointDistributionSequential", batch_shape=[[], [], [], []], event_shape=[[], [5], [], [1000]], dtype=[float32, float32, float32, float32])
